In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

# Turn down for faster convergence
t0 = time.time()
train_samples = 60000

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

print(X.shape)

(70000, 784)


In [7]:
X = X.reshape((X.shape[0], -1))


X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)



scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape)

(60000, 784)


In [8]:
print(X_test.shape)

(10000, 784)


In [9]:
feature_names=[]

for i in range(784):
    feature='feature_'+str(i)
    feature_names.append(feature)
    
feature_names[783]

class_names=[]

for i in range(10):
    class_1='class_'+str(i+1)
    class_names.append(class_1)

X_train[0]

In [10]:
clf = LogisticRegression(C=50. / train_samples, penalty='l1', solver='saga', tol=0.1)
clf.fit(X_train, y_train)

LogisticRegression(C=0.0008333333333333334, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.1, verbose=0,
                   warm_start=False)

In [11]:
#Accuracy on MNIST TestSet after training on the entire data
import sklearn
sklearn.metrics.accuracy_score(y_test, clf.predict(X_test))

0.8366

# Preparation of Seed Data (S)

In [12]:
templist = []
for tup in zip(X_train, y_train):
    if(tup[1]=='8' or tup[1]=='3'):
        templist.append(tup)
        
print(len(templist))

11936


In [17]:
templist = []
for tup in zip(X_train, y_train):
    if(tup[1]=='8' or tup[1]=='3'):
        templist.append(tup)
    
testList = []
for tup in zip(X_test, y_test):
    if(tup[1]=='8' or tup[1]=='3'):
        testList.append(tup)
    
print("Initial Training set size = "+str(len(templist)))
print("Initial Test set size = "+str(len(testList)))

import random
random.seed(0)
random.shuffle(templist)

seed_size=3

seed_list=templist[0:seed_size]
print("Initial Seed set size = "+str(len(seed_list)))
#print(seed_list[0])

Initial Training set size = 11936
Initial Test set size = 2030
Initial Seed set size = 3


# Creating Unlabelled Set (U = X - S)

In [18]:
unlabelled_list=templist[seed_size:]
print(len(unlabelled_list))

11933


In [26]:
type(X_train)

numpy.ndarray

# Creating Explaination vector for seed_set instance

In [25]:
import numpy as np
import lime
import lime.lime_tabular
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_names, class_names=class_names, discretize_continuous=False)

In [20]:
total_np_array=[]

for item in seed_list:
    exp = explainer.explain_instance(item[0], clf.predict_proba, num_features=5)
    vectors = np.empty([1, 5])
    for elem in list(exp.as_list()):
        np.append(vectors, elem[1], axis=None)
    total_np_array.append(vectors[0])

In [21]:
print(total_np_array)

[array([0.0091652 , 0.00877903, 0.00898487, 0.00773941, 0.00756521]), array([0.00931945, 0.00921264, 0.00900133, 0.00835936, 0.00785345]), array([0.00983116, 0.00886553, 0.00833357, 0.00783748, 0.00781561])]


In [22]:
seed_mean = np.mean(np.array(total_np_array), axis=0)

In [23]:
seed_mean

array([0.00943861, 0.0089524 , 0.00877326, 0.00797875, 0.00774476])

In [24]:
len_unlabelled = len(unlabelled_list)
for i in range(len_unlabelled):
    exp = explainer.explain_instance(unlabelled_list[i][0], clf.predict_proba, num_features=5)
    print(exp.as_list())

[('feature_350', 0.010591373801960478), ('feature_406', 0.008570822633616068), ('feature_323', 0.008466110326752358), ('feature_378', 0.008380604960303259), ('feature_322', 0.008059129705455307)]
[('feature_350', 0.009914391605785425), ('feature_378', 0.009490843301822014), ('feature_322', 0.009077754013550751), ('feature_406', 0.007796301771975351), ('feature_323', 0.0074398579665613685)]
[('feature_350', 0.009617896520230753), ('feature_378', 0.008618780127652898), ('feature_322', 0.008165013874584608), ('feature_323', 0.007983306102490829), ('feature_295', 0.007886233569115627)]
[('feature_378', 0.009155381255508325), ('feature_350', 0.008813472828826973), ('feature_489', 0.007855967025193485), ('feature_323', 0.007852361648246183), ('feature_322', 0.00749340069853888)]
[('feature_350', 0.010391459978391483), ('feature_378', 0.008901937145365715), ('feature_322', 0.007994167144950126), ('feature_323', 0.007105389342117292), ('feature_489', 0.007067025548147443)]
[('feature_350', 0.0

KeyboardInterrupt: 

In [34]:
explainer

In [35]:
exp = explainer.explain_instance(X_train[5], clf.predict_proba, num_features=5)

In [36]:
exp.as_list()[0]

('feature_350', 0.0075412721155893045)